In [ ]:
pip install rembg ultralytics

In [ ]:
from pathlib import Path
from rembg import remove
from cv2 import imread, threshold, findContours, RETR_CCOMP, THRESH_BINARY, CHAIN_APPROX_NONE

for path in Path("./datasets/images").glob("*.*"):

    image = imread(str(path))
    alpha = remove(image)[:, :, 3]

    objects = findContours(
        threshold(alpha, 150, 255, THRESH_BINARY)[1],
        RETR_CCOMP,
        CHAIN_APPROX_NONE
    )[0]

    with open(f"./datasets/labels/{path.stem}.txt", "w") as annotations:
        for object in objects:
            contour = object.flatten().tolist()
            coordinates = [(contour[i] / image.shape[1], contour[i + 1] / image.shape[0]) for i in range(0, len(contour), 2)]
            annotations.write(f"0 {' '.join([f'{x:.10f} {y:.10f}' for x, y in coordinates])}\n")

In [ ]:
from ultralytics import YOLO

model = YOLO("./yolov8n-seg.pt")
model.train(data = "./data.yaml")

In [ ]:
model = YOLO("./runs/segment/train/weights/best.pt")
results = model("./test.jpg")